In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import LS_fit

ModuleNotFoundError: No module named 'pandas'

# Prima parte

## Circuito RC

Abbiamo effettuato le misurazioni con il cursore dell'oscilloscopio. \
**Resistenza interna oscilloscopio** = 50 $\Omega$

Intrvallo tensione (0.5,-0.5) V \
Capacità = 10 nF \
resistenza = 67.1 $k\Omega$ \
frequenza = 200 Hz \
incertezza su oscilloscopio (precisione strumento) = 8 mV


In [9]:
# Primo plot dei dati
dati1 = pd.read_csv("Dati1.csv")

tempi1 = dati1["tempo(carica)[ms]"]
carica1 = dati1["tensione(carica1)[mV]"]

# plt.errorbar(tempi1, carica1, [8 for i in carica1], fmt = '.')
# plt.show()

In [10]:
# prova fit dei dati con libreria
def mod_exp(x, A, T):
    return A*np.exp(-x/T)

par1, val1, ers1, pval1, *other = LS_fit(tempi1, carica1, [8 for i in carica1], mod_exp, disp=1, A=1,T=1) # type: ignore

for p, v, e in zip(par1, val1, ers1):
    print(f"{p} = {v:.3f} ± {e:.3f}")

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.982 (χ²/ndof = 0.2)      │              Nfcn = 79               │
│ EDM = 1.33e-07 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ A    │    962    │     6     │            │            │         │         │       │
│ 1 │ T    │   64.8    │    0.6    │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬─────────────┐
│   │     A     T │
├───┼─────────────┤
│ A │  31.3  -2.4 │
│ T │  -2.4 0.406 │
└───┴─────────────┘

A = 962.243 ± 5.590
T = 64.769 ± 0.637


## Circuito RL

induttanza circa 10 mH \
resistenza = 1 $k\Omega$ \
frequenza = 700 Hz \
range di tensione = -1,1 V \
precisione oscilloscopio = 20 mV

In [12]:
# Fit circuito RL

def mod_scarica(x, A, T):
    return A * (1-np.exp(-x/T))

dati2 = pd.read_csv("Dati2.csv")
tempi2 = dati2["tempo"]
carica2 = dati2["tensione(carica1)"]

par2, val2, ers2, pval2, *other = LS_fit(tempi2, carica2, [8 for i in carica2], mod_scarica, disp=1, A=1,T=1) # type: ignore

for p, v, e in zip(par2, val2, ers2):
    print(f"{p} = {v:.3f} ± {e:.3f}")

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.211e+04 (χ²/ndof = 1578.9)│              Nfcn = 116              │
│ EDM = 1.24e-06 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ A    │  1.69e3   │  0.04e3   │            │            │         │         │       │
│ 1 │ T    │   0.223   │   0.007   │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬─────────────────────┐
│   │         A         T │
├───┼─────────────────────┤
│ A │  1.78e+03 312.46e-3 │
│ T │ 312.46e-3   5.6e-05 │
└───┴─────────────────────┘

A = 1692.002 ± 42.189
T = 0.223 ± 0.007


 # Seconda parte

## Circuito RLC

Resistenza = 300 $\Omega$ \
Induttanza e capacità uguali a prima \
Intervallo tensione (1,0)

Abbiamo stimato le costanti del sistema prima di prendere i dati:
* $\omega_0$ = 100 kHz
* $f_0 = \frac{\omega_0}{2\pi}$ = 16kHz 

Ci siamo resi presto conto che questi risultati sono insensati \
Frequenza effettivamente usata = 300 Hz


Per smorzamento critico abbiamo usato Resistenza = 3900 $\Omega$ \
Per sovrasmorzamento abbiamo usato Resistenza = 10.000 $\Omega$